In [1]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalAveragePooling2D, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from skimage.transform import resize

import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only allocate 1GB * 2 of memory on the first GPU
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024 * 2)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)

Using TensorFlow backend.


1 Physical GPUs, 1 Logical GPUs


In [2]:
NUM_CLASSES = 10
BATCH_SIZE = 5
NUM_EPOCHS = 10
use_data_aug = False

In [3]:
from tqdm import tqdm
# img_arr is of shape (n, h, w, c)
def resize_image_arr(img_arr):
    x_resized_list = []
    for i in tqdm(range(img_arr.shape[0])):
        img = img_arr[0]
        resized_img = resize(img, (224, 224))
        x_resized_list.append(resized_img)
    return np.stack(x_resized_list)




In [15]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [5]:
import pandas as pd

x_df= x_test.reshape(10000, x_test.shape[1] * x_test.shape[2] *x_test.shape[3] )

imgs = pd.DataFrame() 
df_labels = pd.DataFrame() 

imgs = imgs.append(pd.DataFrame(x_df)) 
df_labels = df_labels.append(pd.DataFrame(y_test)) 
imgs['labels'] = df_labels 

imgs

,0,1,2,3,4,5,6,7,8,9,...,3063,3064,3065,3066,3067,3068,3069,3070,3071,labels
0,158,112,49,159,111,47,165,116,51,166,...,24,77,124,34,84,129,21,67,110,3
1,235,235,235,231,231,231,232,232,232,232,...,168,183,178,180,195,191,186,200,199,8
2,158,190,222,158,187,218,139,166,194,132,...,5,6,8,4,5,3,7,8,7,8
3,155,156,149,167,176,187,176,179,193,190,...,60,63,50,64,65,52,73,68,50,0
4,65,68,50,70,81,64,48,64,46,30,...,143,179,136,154,185,146,128,156,117,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,76,78,133,76,78,134,77,80,132,76,...,204,219,236,200,215,234,199,216,236,8
9996,81,57,43,91,69,53,98,75,63,106,...,155,139,129,171,167,147,167,164,160,3
9997,20,15,12,19,14,11,15,14,11,15,...,24,23,50,26,23,53,25,20,47,5
9998,25,40,12,15,36,3,23,41,18,17,...,93,124,80,95,123,81,92,120,80,1


In [6]:
x_test = resize_image_arr(x_test)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:35<00:00, 104.86it/s]


In [16]:
# Convert class vectors to binary class matrices.
#y_test = keras.utils.to_categorical(y_test, NUM_CLASSES)

# Normalize the data
x_test = x_test.astype('float32')
x_test /= 255

In [17]:
np.min(x_test),np.max(x_test)

(0.0, 1.0)

In [18]:
base_model = ResNet50(include_top=False, weights= None)

C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\keras_applications\resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
img_idx = imgs.loc[imgs['labels'] == i][:100]

abc = img_idx.iloc[:, :3072]

for idx, row in abc.iterrows():
    print(idx)

In [12]:
x_test.shape

(10000, 224, 224, 3)

In [53]:
from sklearn.preprocessing import normalize, scale, MinMaxScaler

(x_train_SAMP, y_train_SAMP), (x_test_SAMP, y_test_SAMP) = cifar10.load_data()

x_train_SAMP =[]
y_train_SAMP =[]

result = {}
LAYER_COUNTER = 0
for idx in range(len(base_model.layers)):

    
    if base_model.get_layer(index = idx).__class__.__name__ == 'Conv2D':
        # PREPARE FOR JSON 
        result['dataset_'+str(LAYER_COUNTER)]= []
    
        for i in range(0,10):
            
            
            img_idx = imgs.loc[imgs['labels'] == i][:100]
            
            temp_img_arr = []
            for index, row in img_idx.iloc[:, :3072].iterrows():
                #print(x_test_SAMP[index].shape)
                #Xtest = np.reshape(data[b"data"][3], (-1, 3, 32, 32))
                #Xtest = np.transpose(Xtest, (0,2,3,1))

                #x1 = np.reshape(np.asarray(row),(-1,32,32,3))
                #x1 = np.transpose(x1, (0,2,3,1))
                temp_img_arr.append(x_test[index])
            
            #print(temp_img_arr)
            #temp_img_arr = np.array(temp_img_arr).reshape(-1,32,32,3)
            temp_img_arr = np.stack(temp_img_arr,axis=0)
            print(temp_img_arr.shape)
            
            x_batch = resize_image_arr(temp_img_arr)
            print(x_batch.shape)

            x_batch = x_batch.astype('float32')
            x_batch /= 255
            
            
            layerName = base_model.get_layer(index = idx).name
            print(idx, layerName)

            model_cut = Model(inputs=base_model.inputs, output=base_model.layers[idx].output)
            preds = model_cut.predict(x_batch)
            print('preds',preds.shape)

            preds_flat = preds.reshape(preds.shape[0],preds.shape[1] * preds.shape[2] * preds.shape[3])
            print(preds_flat.shape)

            import pandas as pd 

            X = preds_flat
            # Standardize the features
            #X = StandardScaler().fit_transform(preds_flat)
            #X = normalize(preds_flat)
            #X = scale(preds_flat)

            #mm_scaler = MinMaxScaler()
            #X = mm_scaler.fit_transform(preds_flat)
            #mm_scaler.transform(X)

            # Preview X
            pd.DataFrame(data=X).head()

            # Import PCA from sklearn
            from sklearn.decomposition import PCA

            # Instantiate PCA
            pca = PCA(n_components=2)

            # Fit PCA to features
            principalComponents = pca.fit_transform(X)
            
            CLASS_PCA_RES = []
            for ipc in range(principalComponents.shape[0]):
                arr_classPt = {
                    "uid" : "",
                    "filename" : "",
                    "comp1": str(principalComponents[ipc][0]),
                    "comp2": str(principalComponents[ipc][1])
                }
                CLASS_PCA_RES.append(arr_classPt)
                
            result["dataset_"+str(LAYER_COUNTER)].append(CLASS_PCA_RES)

            # Create a new dataset from principal components 
            df = pd.DataFrame(data = principalComponents, 
                              columns = ['PC1', 'PC2'])

            target = pd.Series(y_test_SAMP[:100].reshape(100,), name='target')

            result_df = pd.concat([df, target], axis=1)
            print(result_df)
        
        LAYER_COUNTER+= 1
    if idx > 50:
        break


 11%|██████████▌                                                                                     | 11/100 [00:00<00:00, 104.05it/s]

(100, 32, 32, 3)


100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 104.01it/s]
C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("co...)`


(100, 224, 224, 3)
2 conv1
preds (100, 112, 112, 64)
(100, 802816)


C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\sklearn\decomposition\_pca.py:555: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ / total_var.sum()
 12%|███████████▌                                                                                    | 12/100 [00:00<00:00, 115.69it/s]

...

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 116.21it/s]
C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("re...)`


(100, 224, 224, 3)
51 res3b_branch2a
preds (100, 28, 28, 128)
(100, 100352)
         PC1           PC2  target
0   0.000005 -8.755050e-08       3
1   0.000005 -8.754949e-08       8
2   0.000005 -8.754961e-08       8
3   0.000005 -8.754961e-08       0
4   0.000005 -8.754961e-08       6
..       ...           ...     ...
95  0.000005 -8.754961e-08       6
96  0.000005  2.096427e-06       6
97  0.000005  2.096427e-06       0
98  0.000005  2.096427e-06       0
99  0.000005  2.096427e-06       7

[100 rows x 3 columns]
(100, 32, 32, 3)

 24%|███████████████████████                                                                         | 24/100 [00:00<00:00, 117.72it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 113.04it/s]
C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:47: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("re...)`


(100, 224, 224, 3)
51 res3b_branch2a
preds (100, 28, 28, 128)
(100, 100352)
         PC1           PC2  target
0   0.000004 -7.831431e-08       3
1   0.000004 -7.831368e-08       8
2   0.000004 -7.831407e-08       8
3   0.000004 -7.831407e-08       0
4   0.000004 -7.831407e-08       6
..       ...           ...     ...
95  0.000004 -7.831407e-08       6
96  0.000004  1.875508e-06       6
97  0.000004  1.875508e-06       0
98  0.000004  1.875508e-06       0
99  0.000004  1.875508e-06       7

[100 rows x 3 columns]


In [ ]:
x_test[:10].shape

In [ ]:
from sklearn.preprocessing import normalize, scale, MinMaxScaler

(x_train_SAMP, y_train_SAMP), (x_test_SAMP, y_test_SAMP) = cifar10.load_data()

x_train_SAMP =[]
y_train_SAMP =[]



for idx in range(len(base_model.layers)):
    if base_model.get_layer(index = idx).__class__.__name__ == 'Conv2D':
        for i in range(0,10):
            x_batch = x_test[:100]
            
           
            
            layerName = base_model.get_layer(index = idx).name
            print(idx, layerName)

            model_cut = Model(inputs=base_model.inputs, output=base_model.layers[idx].output)
            preds = model_cut.predict(x_batch)
            print('preds',preds.shape)

            preds_flat = preds.reshape(preds.shape[0],preds.shape[1] * preds.shape[2] * preds.shape[3])
            print(preds_flat.shape)

            import pandas as pd 

            X = preds_flat
            # Standardize the features
            #X = StandardScaler().fit_transform(preds_flat)
            #X = normalize(preds_flat)
            #X = scale(preds_flat)

            #mm_scaler = MinMaxScaler()
            #X = mm_scaler.fit_transform(preds_flat)
            #mm_scaler.transform(X)

            # Preview X
            pd.DataFrame(data=X).head()

            # Import PCA from sklearn
            from sklearn.decomposition import PCA

            # Instantiate PCA
            pca = PCA(n_components=2)

            # Fit PCA to features
            principalComponents = pca.fit_transform(X)

            # Create a new dataset from principal components 
            df = pd.DataFrame(data = principalComponents, 
                              columns = ['PC1', 'PC2'])

            target = pd.Series(y_test_SAMP[:10].reshape(10,), name='target')

            result_df = pd.concat([df, target], axis=1)
            print(result_df)
        
    if idx > 10:
        break


In [22]:
from sklearn.preprocessing import StandardScaler
import pandas as pd 

from sklearn.preprocessing import normalize
# Standardize the features
#X = StandardScaler().fit_transform(preds_flat)

# Preview X
pd.DataFrame(data=preds_flat).head()

# Import PCA from sklearn
from sklearn.decomposition import PCA

# Instantiate PCA
pca = PCA(n_components=2)

# Fit PCA to features
principalComponents = pca.fit_transform(preds_flat)

C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\sklearn\decomposition\_pca.py:555: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ / total_var.sum()


In [52]:
from sklearn.preprocessing import StandardScaler

img_idx = imgs.loc[imgs['labels'] == 0][:100]
            
temp_img_arr = []
for index, row in img_idx.iloc[:, :3072].iterrows():
    temp_img_arr.append(x_test[index])

temp_img_arr = np.stack(temp_img_arr,axis=0)
print(temp_img_arr.shape)

layerName = base_model.get_layer(index = idx).name
print(idx, layerName)

model_cut = Model(inputs=base_model.inputs, output=base_model.layers[idx].output)
preds = model_cut.predict(x_batch)
print('preds',preds.shape)

preds_flat = preds.reshape(preds.shape[0],preds.shape[1] * preds.shape[2] * preds.shape[3])
print(preds_flat.shape)

feat_cols = ['pixel'+str(i) for i in range(preds_flat.shape[1])]
df_cifar = pd.DataFrame(preds_flat,columns=feat_cols)

df_cifar.head()

import pandas as pd 


X = preds_flat
# Standardize the features
#X = StandardScaler().fit_transform(df_cifar.values.reshape)
#x_min = X.min(axis=(0, 1), keepdims=True)
#x_max = X.max(axis=(0, 1), keepdims=True)

#X = (X - x_min)/(x_max-x_min)

with np.printoptions(threshold=np.inf):
    #print(arr)
    print(preds[0])
    print(preds[1])
#print(X[1])
#print(X.shape)
#X = normalize(preds_flat)
#X = scale(preds_flat)

#mm_scaler = MinMaxScaler()
#X = mm_scaler.fit_transform(preds_flat)
#mm_scaler.transform(X)

# Preview X
pd.DataFrame(data=X).head()

# Import PCA from sklearn
from sklearn.decomposition import PCA

# Instantiate PCA
pca = PCA(n_components=2)

# Fit PCA to features
principalComponents = pca.fit_transform(X)

CLASS_PCA_RES = []
for ipc in range(principalComponents.shape[0]):
    arr_classPt = {
        "uid" : "",
        "filename" : "",
        "comp1": str(principalComponents[ipc][0]),
        "comp2": str(principalComponents[ipc][1])
    }
    #print(arr_classPt)



# Create a new dataset from principal components 
df = pd.DataFrame(data = principalComponents, 
                  columns = ['PC1', 'PC2'])

target = pd.Series(y_test_SAMP[:100].reshape(100,), name='target')

result_df = pd.concat([df, target], axis=1)
#print(result_df)

(100, 32, 32, 3)
21 activation_103


C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("ac...)`
  from ipykernel import kernelapp as app


preds (100, 56, 56, 64)
(100, 200704)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

C:\Users\benza-ord\anaconda3\envs\keras-gpu\lib\site-packages\sklearn\decomposition\_pca.py:555: RuntimeWarning: divide by zero encountered in true_divide
  self.explained_variance_ / total_var.sum()


In [28]:
feat_cols = ['pixel'+str(i) for i in range(preds_flat.shape[1])]
df_cifar = pd.DataFrame(preds_flat,columns=feat_cols)

df_cifar.values

(100, 200704)

In [ ]:
result

In [54]:
import json 

result["n_layers"] = str(LAYER_COUNTER)

with open('sample_json4.json', 'w') as outfile:
    json.dump(result, outfile)

In [21]:
np.min(x_test),np.max(x_test)

(0.0, 1.0)